In [196]:
import numpy as np
import gensim as gn
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string
from scipy.sparse import csc_matrix, save_npz
import json
import os, re, math
import multiprocessing as mp
import pandas as pd

In [109]:
wc = load_dict("wc1.csv")

In [110]:
wnd = load_dict("wnd1.csv")

In [108]:
def load_dict(file):
    data = {}
    df = pd.read_csv(file)
    data = df.to_dict()
    return data

In [220]:
def sum_dict(d):
    s = 0
    for key in d:
        s = s + d[key][0]
    return s

def create_PPI(wc, wnd):
    len_d = sum_dict(wc)
    len_wc = len(wc)
    PMI = csc_matrix((len_wc, len_wc),dtype=float) # scipy sparse matrix
    for key, y_ind in enumerate(wc.keys()):
        for key1, x_ind in enumerate(wc.keys()):
            s = (key,key1)
            try: 
                w = wnd[str(s)][0]
                t = w * len_d
                b = wc[key][0] * wc[key1][0]
                fin = math.log(t/b)
                PMI[x_ind, y_ind] = fin
                print("PMI[",x_ind,",",y_ind,"] = ",fin)
            except KeyError:
                continue
    return PMI

In [ ]:
PMI = create_PMI(wc,wnd)